<a href="https://colab.research.google.com/github/AishJo/Repository1/blob/master/LSTM_Glove_Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!pip install -U imbalanced-learn

In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases
import re 
from gensim.parsing.preprocessing import remove_stopwords
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [4]:
%%time
# reading the data
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
# creating a subset of the relevant attribute name and then dropping the column
df = df[df['attribute_name'] == 'category'].drop(columns = ['attribute_name'])
# combining similar category attribute values and removing spaces
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'sweatshirthoodie' if x == 'sweatshirt hoodie' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'blazerscoatsjackets' if x == 'blazer , coat jacket' else x)

CPU times: user 369 ms, sys: 27.8 ms, total: 397 ms
Wall time: 428 ms


In [5]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
3,01DVCVGDJQC4J8JJ35PKZ4J169,loewe,hammock small textured leather shoulder bag,black textured leather calf hook fasten open w...,bag tote bag tote bag,accessory,item 's measurement depth 14 cm height 23 cm m...
32,01DPGSTG4M1RXB26QMMN0MPPB8,j crew,mockneck sweater supersoft yarn,cozy flattering mockneck sweater feel good loo...,sweater,sweater,NaN
36,01E2KKRH99SZ1PTR2E2ABVYJ5S,alo,vapor high waist legging,signature lift fabric moisture wicking technol...,unknown,NaN,"xxs 00 0 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 ,..."
50,01E2M1MYJ1FYVHG2AMXHDMNQSP,alo,prism high waist capris,high waistband help flat figure moisture wicki...,unknown,NaN,"xxs 00 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 hig..."
52,01E2KZ8BZF6M1XCHBZGEX3AF78,alo,cover tank,hard earn abs supersoft faux wrap tank 's crop...,unknown,NaN,"xxs 00 0 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 ,..."


In [0]:
# creating a feature combining brand, productname, description and brand category
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
# the predictor 'text' is assigned to 'X' and the 'attribute_value' t 'y'
X = df['text'].values
# one-hot-encoding the y variable
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
# creating a function to create uni-grams and tokenizing data
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X, mode)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
# creating a function to create bi-grams and tokenizing data
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X, mode)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
#Tokenizer.texts_to_matrix()

In [0]:
# creating tf-idf vectors from uni-gram encoding
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [0]:
# creating tf-idf vectors from bi-gram encoding
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [0]:
#X1 = X1.reshape(-1, 155, 1)
#X2 = X2.reshape(-1, 155, 1)

In [73]:
X1.shape

(9975, 199)

In [0]:
# concatenating uni-gram and bi-gram
X = np.concatenate([X1, X2], axis = 1)

In [0]:
# splitting data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
# balancing the sample sets as the proportion of labels is unbalanced
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
# defining LSTM model with sigmoid activation functions for prediction
def define_model():
    inputs = Input(shape=(length1+length2,))
    #x1 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs1)
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    #inputs2 = Input(shape=(length2,))
    #x2 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs2)
    #dense2 = Dense(100, activation='relu')(inputs2)
    #embedding2 = Embedding(input_dim=vocab_size2, output_dim=100)(inputs2)

    #merged = concatenate([inputs1, inputs1])
    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    #x = Dense(100, activation = 'relu')(inputs)


    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=length1+length2, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    #x = Dense(num_classes)(x)
    out = Activation('sigmoid')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [79]:
model = define_model()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 398)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 398, 100)          595700    
_________________________________________________________________
lstm_3 (LSTM)                (None, 398, 16)           7488      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 398, 7)            119       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1, 7)              19509     
_________________________________________________________________
reshape_3 (Reshape)          (None, 7)                 0         
_________________________________________________________________
activation_3 (Activation)    (None, 7)                 0   

In [80]:
# fitting data onto model
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 17039 samples, validate on 4260 samples
Epoch 1/10
17039/17039 [==============================] - 26s 2ms/step - loss: 0.4310 - accuracy: 0.8470 - val_loss: 0.7176 - val_accuracy: 0.8571
Epoch 2/10
17039/17039 [==============================] - 26s 2ms/step - loss: 0.3909 - accuracy: 0.8571 - val_loss: 0.7612 - val_accuracy: 0.8571
Epoch 3/10
17039/17039 [==============================] - 26s 1ms/step - loss: 0.3894 - accuracy: 0.8571 - val_loss: 0.7385 - val_accuracy: 0.8571
Epoch 4/10
17039/17039 [==============================] - 25s 1ms/step - loss: 0.3877 - accuracy: 0.8571 - val_loss: 0.7345 - val_accuracy: 0.8571
Epoch 5/10
17039/17039 [==============================] - 25s 1ms/step - loss: 0.3817 - accuracy: 0.8571 - val_loss: 0.7210 - val_accuracy: 0.8571
Epoch 6/10
17039/17039 [==============================] - 26s 1ms/step - loss: 0.3703 - accuracy: 0.8567 - val_loss: 0.7189 - val_accuracy: 0.8530
Epoch 7/10
17039/17039 [==============================] - 26s 2ms/ste

In [81]:
# evaluating model
model.evaluate(X_test, y_test)

998/998 [==============================] - 2s 2ms/step


[0.3710651554779443, 0.8541370034217834]

In [0]:
# probability of predicting each class for various records
results = model.predict(X)

In [0]:
#Creating a DataFrame for the results
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [84]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [85]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,coldweather,daytonight,nightout,vacation,weekend,work,workout,attribute_value
0,,daytonight,nightout,,weekend,work,,"daytonight, nightout, weekend, work"
1,coldweather,daytonight,nightout,vacation,weekend,,,"coldweather, daytonight, nightout, vacation, w..."
2,coldweather,daytonight,,,weekend,,,"coldweather, daytonight, weekend"
3,coldweather,daytonight,nightout,vacation,weekend,,,"coldweather, daytonight, nightout, vacation, w..."
4,coldweather,daytonight,nightout,vacation,weekend,,,"coldweather, daytonight, nightout, vacation, w..."


In [86]:
results_df['attribute_value'].value_counts()

coldweather, daytonight, nightout, vacation, weekend          2744
daytonight, nightout, weekend, work                           1881
daytonight, nightout, weekend                                 1222
daytonight, nightout, vacation, weekend                        751
daytonight, nightout, vacation, weekend, work                  543
daytonight, weekend, work                                      492
daytonight, vacation, weekend                                  488
daytonight, weekend                                            362
coldweather, daytonight, nightout, weekend, work               326
coldweather, daytonight, nightout, vacation, weekend, work     295
coldweather, daytonight, weekend, work                         235
coldweather, daytonight, nightout, weekend                     232
daytonight, vacation, weekend, work                            216
coldweather, daytonight, weekend                                68
daytonight, weekend, work, workout                            

In [0]:
# tokenizing the data and integer encoding it
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
# running the tokenizer function on features
X = tokenizer.texts_to_sequences(X)

In [0]:
# getting vocab size and length for input
length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
# padding the data to make it same size (max size)
X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
# splitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [13]:
# balancing the dataset to account for disproportionate of labels
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [0]:
num_classes = y.shape[1]

In [16]:
# using glove vector to create function that makes word emebeddings
def load_glove_vectors():
    embeddings_index = {}
    with open('/content/drive/Shared drives/DSO 560 NLP Project/glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
# creating an LSTM model
def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    #model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    #model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [21]:
model = define_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 155, 100)          615100    
_________________________________________________________________
masking_2 (Masking)          (None, 155, 100)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 155, 64)           42240     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 119       
Total params: 691,523
Trainable params: 76,423
Non-trainable params: 615,100
___________________________________________

In [20]:
# fitting model on training data
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 11055 samples, validate on 2764 samples
Epoch 1/10
11055/11055 [==============================] - 19s 2ms/step - loss: 0.6397 - accuracy: 0.7459 - val_loss: 0.6130 - val_accuracy: 0.8571
Epoch 2/10
11055/11055 [==============================] - 17s 1ms/step - loss: 0.5865 - accuracy: 0.8569 - val_loss: 0.5567 - val_accuracy: 0.8571
Epoch 3/10
11055/11055 [==============================] - 16s 1ms/step - loss: 0.5079 - accuracy: 0.8571 - val_loss: 0.5308 - val_accuracy: 0.8571
Epoch 4/10
11055/11055 [==============================] - 16s 1ms/step - loss: 2.7004 - accuracy: 0.8567 - val_loss: 81.4542 - val_accuracy: 0.8536
Epoch 5/10
11055/11055 [==============================] - 16s 1ms/step - loss: 31.0084 - accuracy: 0.8385 - val_loss: 63.6311 - val_accuracy: 0.8239
Epoch 6/10
11055/11055 [==============================] - 16s 1ms/step - loss: 233410.5102 - accuracy: 0.8111 - val_loss: 5623585.1751 - val_accuracy: 0.7582
Epoch 7/10
11055/11055 [==============================]

In [23]:
# evaluating model
model.evaluate(X_test, y_test)

420/420 [==============================] - 1s 4ms/step


[0.6990222726549421, 0.42414966225624084]

In [0]:
# making predictions
results = model.predict(X)

In [0]:
# defining a threshold for prediction
results_df = pd.DataFrame()
results_mask = results > 0.5
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [48]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [49]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,accessory,blazerscoatsjackets,onepiece,piece,shoe,sweater,sweatshirthoodie,attribute_value
0,accessory,blazerscoatsjackets,,piece,,sweater,,"accessory, blazerscoatsjackets, piece, sweater"
1,accessory,blazerscoatsjackets,,piece,,sweater,sweatshirthoodie,"accessory, blazerscoatsjackets, piece, sweater..."
2,accessory,blazerscoatsjackets,,piece,,sweater,,"accessory, blazerscoatsjackets, piece, sweater"
3,accessory,blazerscoatsjackets,,piece,,sweater,,"accessory, blazerscoatsjackets, piece, sweater"
4,accessory,blazerscoatsjackets,,piece,,sweater,,"accessory, blazerscoatsjackets, piece, sweater"


In [50]:
results_df['attribute_value'].value_counts()

accessory, blazerscoatsjackets, piece, sweater                                      2048
accessory, piece, shoe, sweatshirthoodie                                             606
accessory, piece, sweater                                                            457
accessory, blazerscoatsjackets, piece, sweater, sweatshirthoodie                     327
accessory, blazerscoatsjackets, piece                                                194
accessory, blazerscoatsjackets, sweater                                              111
accessory, blazerscoatsjackets, piece, sweatshirthoodie                               95
accessory, blazerscoatsjackets, sweater, sweatshirthoodie                             74
accessory, blazerscoatsjackets, piece, shoe, sweater, sweatshirthoodie                59
accessory, piece, sweater, sweatshirthoodie                                           49
accessory, blazerscoatsjackets                                                        37
accessory, piece     

In [0]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
def get_style(brand, product_full_name, description, details, brand_category):
    def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''

    def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])
        
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('model.pickle', 'rb') as handle:
        model = pickle.load(handle)

    data = pd.DataFrame({'brand':brand,'product_full_name':product_full_name,'description':description,'details':details,'brand_category':brand_category},index=[0]) 
    df = data.copy()
    for key in df.columns:
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
        df[key] = df[key].apply(lemmatizer)
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
    df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)
    X = df['text'].values
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    results = model.predict(X)
    results_df = pd.DataFrame()
    results_mask = results > 0.1
    for i in range(len(label_list)):
        results_df[label_list[i]] = results_mask[:,i]
        results_df[label_list[i]] = results_df[label_list[i]].apply(int)
    results_df['sum']  = 0
    for key in label_list:
        results_df['sum'] = results_df['sum'] + results_df[key]
    (results_df['sum'] == 0).sum()
    results_df = results_df.drop(columns = ['sum'])
    for key in label_list:
        results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

    results_df['attribute_value'] = ''
    for key in label_list:
        results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

    results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
    data['attribute_value'] = results_df['attribute_value']
    return data

In [0]:
brand = "frame"
product_full_name = "les second medium noir"
description = "'minimal , modern styling meet refined luxury les second caba tote craft exquisite leather , structured handbag equip adjustable high polish peg buttonhole double drop handle , center welt seam , detachable pouch , frame logo discreetly emboss'"
details = np.nan
brand_category = 'accessory'

In [54]:
get_style(brand, product_full_name, description, details, brand_category)

,brand,product_full_name,description,details,brand_category,attribute_value
0,frame,les second medium noir,"'minimal , modern styling meet refined luxury ...",NaN,accessory,"accessory, blazerscoatsjackets, onepiece, piec..."
